# Global Parameter Optimization

In [1]:
# Activate local environment, see `Project.toml`
import Pkg; Pkg.activate("."); Pkg.instantiate(); 

  Activating project at `~/.julia/dev/RxInfer/examples`


This notebook demonstrates how to optimize the parameters of the state-space model (implemented in **RxInfer.jl**) through an external optimization packages such as [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl/)

We use the following model:
$$\begin{aligned}
    {x}_t &= {x}_{t-1} + c \\
    {y}_t &\sim \mathcal{N}\left({x}_{t}, p \right) 
\end{aligned}$$
with prior ${x}_0 \sim \mathcal{N}({m_{{x}_0}}, {v_{{x}_0}})$. Our goal is to optimize parameters $c$ and ${m_{{x}_0}}$.

In [2]:
using RxInfer, BenchmarkTools, Random

In [3]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

In [4]:
rng = MersenneTwister(42)

P      = 1.0
n      = 250
c_real = -5.0
data   = c_real .+ collect(1:n) + rand(rng, Normal(0.0, sqrt(P)), n);

In [8]:
# c[1] is C
# c[2] is μ0
function f(c)
    x0_prior = NormalMeanVariance(c[2], 100.0)
    result = inference(
        model = smoothing(n, x0_prior, c[1], P), 
        data  = (y = data,), 
        free_energy = true
    )
    return result.free_energy[end]
end

f (generic function with 1 method)

In [9]:
using Optim

In [10]:
res = optimize(f, ones(2), GradientDescent(), Optim.Options(g_tol = 1e-3, iterations = 100, store_trace = true, show_trace = true, show_every = 10))

Iter     Function value   Gradient norm 
     0     3.683766e+02     7.580911e+02
 * time: 0.019810199737548828
    10     3.679681e+02     2.277267e+01
 * time: 5.799490213394165


 * Status: success

 * Candidate solution
    Final objective value:     3.679676e+02

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 5.09e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.69e-05 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 4.59e-08 ≰ 0.0e+00
    |g(x)|                 = 9.57e-04 ≤ 1.0e-03

 * Work counters
    Seconds run:   10  (vs limit Inf)
    Iterations:    19
    f(x) calls:    142
    ∇f(x) calls:   142


In [11]:
res.minimizer # Real values are indeed (c = 1.0 and μ0 = -5.0)

2-element Vector{Float64}:
  1.0005879774879456
 -5.043988393663863

In [12]:
println("Real value vs Optimized")
println("Real:      ", [ 1.0, c_real ])
println("Optimized: ", res.minimizer)

Real value vs Optimized
Real:      [1.0, -5.0]
Optimized: [1.0005879774879456, -5.043988393663863]
